In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as wg
from scipy.signal import find_peaks
import sys
sys.path.append("../src")
import importlib
import support
import visualizations
import calibrations
import kinematics
from scipy import signal
importlib.reload(support)
from support import readFileDialog, parse_data_file, getFrame, writeToDATA, saveFileDialog
importlib.reload(kinematics)
from kinematics import getKeypointsCoord, getKeypointsPixels, inverseKinematicsRowing
importlib.reload(visualizations)
from visualizations import showFrame, keypointsDATAtoFrame
importlib.reload(calibrations)
from calibrations import getMmppInterface, defineReference
%matplotlib widget

In [34]:
file_path_op = readFileDialog("Open Data file (OP)", "data")
file_path_vicon = readFileDialog("Open Data file (Vicon)", "data")

In [35]:
data, var_names = parse_data_file(file_path_op)
kp_openpose = data["keypoints"]
md_openpose = data["metadata"]
ang_openpose = data["angles"]
fps_openpose = md_openpose["fps"]
angles_names = md_openpose['angles_names']
t_openpose = np.linspace(0, len(ang_openpose)*(1/fps_openpose), len(ang_openpose))

In [36]:
data, var_names = parse_data_file(file_path_vicon)
kp_vicon = data["keypoints"]
md_vicon = data["metadata"]
ang_vicon = data["angles"]
fps_vicon = md_vicon["fps"]
angles_names = md_vicon['angles_names']
n_frames = md_vicon['n_frames']
t_vicon = np.linspace(0, len(ang_vicon)*(1/fps_vicon), len(ang_vicon))

In [37]:
sync_correction_sec = 2.3
sync_correction_frames = int(round(sync_correction_sec * fps_vicon))
kp_vicon_sync = kp_vicon[sync_correction_frames:]
ang_vicon_sync = ang_vicon[sync_correction_frames:]
n_frames_sync = n_frames - sync_correction_frames
t_vicon_sync = np.linspace(0, n_frames_sync*(1/fps_vicon), n_frames_sync)

In [38]:
joint_angle = wg.Dropdown(options=angles_names,
                        description='Angle:',
                        value='Shoulder <- Hip -> Knee',
                        disabled=False)

def plot_2d_comparison(joint_angle):
    angle_idx = angles_names.index(joint_angle)
    plt.close('all')
    plt.figure()
    plt.plot(t_vicon_sync, ang_vicon_sync[:, angle_idx], label='Vicon')
    plt.plot(t_openpose, ang_openpose[:, angle_idx],  label='Openpose')
    angle_name = joint_angle.split('->')[0].split('<-')[-1].replace(" ", "")
    plt.title("2D Comparation for the {}".format(angle_name))
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (Degrees)")
    plt.grid(True)
    plt.legend()
    plt.show()

out = wg.interactive_output(plot_2d_comparison, {'joint_angle': joint_angle})

vbox = wg.VBox([joint_angle, out])
display(vbox)

In [39]:
md_vicon_sync = md_vicon.copy()
md_vicon_sync['n_frames'] = n_frames_sync

In [40]:
output_path = saveFileDialog()

In [41]:
output_path

'C:/Users/victo/Documents/Motion_analysis/Vicon/Celio_Vicon/Voga24_5min_Celio_sync'

In [42]:
writeToDATA(output_path, md_vicon_sync, write_mode='w')
for i in range(len(kp_vicon_sync)):
    file_data = {'keypoints': kp_vicon_sync[i].tolist(),
                 'angles': ang_vicon_sync[i].tolist()}
    writeToDATA(output_path, file_data, write_mode='a')